API Python -Challenge: Part 2: VacationPy Student: Jorge Alberto Muñozcano Castro

# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [15]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
# Import API key
url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [16]:
#Extract the new dataframe that was created in the WeatherPy challenge
cityweatherforcast = pd.read_csv("../output_data/cityweatherforcast.csv")
cityweatherforcast.head() #test succesfully data frame generated

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Latitude,Longitude,Temp,Winds Speed
0,0,Berlevåg,75,NO,1585581254,79,70.86,29.09,21.20,11.41
1,1,Punta Arenas,75,CL,1585581254,66,-53.15,-70.92,50.38,23.04
2,2,New Norfolk,50,AU,1585581254,62,-42.78,147.06,50.29,6.93
3,3,Norman Wells,5,CA,1585581254,74,65.28,-126.83,-27.40,8.05
4,4,Celestún,0,MX,1585581255,71,20.87,-90.40,80.69,9.46


In [17]:
#Eliminate duplicate columns in the Data Frame
vacationweatherforcast = cityweatherforcast[["City", "Cloudiness", "Country","Date", "Humidity",
                                             "Latitude","Longitude", "Temp", "Winds Speed"]]
vacationweatherforcast.head()#Test Succesfully Dataframe generated

,City,Cloudiness,Country,Date,Humidity,Latitude,Longitude,Temp,Winds Speed
0,Berlevåg,75,NO,1585581254,79,70.86,29.09,21.20,11.41
1,Punta Arenas,75,CL,1585581254,66,-53.15,-70.92,50.38,23.04
2,New Norfolk,50,AU,1585581254,62,-42.78,147.06,50.29,6.93
3,Norman Wells,5,CA,1585581254,74,65.28,-126.83,-27.40,8.05
4,Celestún,0,MX,1585581255,71,20.87,-90.40,80.69,9.46


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [18]:
#Stablished the max variables of Humidity and locations using previouse dataframe
vacationhumidity = vacationweatherforcast["Humidity"].astype(float)
maxvacationhumidity = vacationhumidity.max()
locations = vacationweatherforcast[["Latitude","Longitude" ]]

In [19]:
#Generated Heat map by on gmaps, using the data frame from previouse dataframe, and give a littleformat to the heatmap
Heatmap_layer = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=vacationhumidity,dissipating=True, 
                                 max_intensity=maxvacationhumidity,point_radius=4)
Heatmap_layer.add_layer(heat_layer)
print(f"Heatmap of all the cities")
Heatmap_layer#test succesfull heatmap generated

Heatmap of all the cities


Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [23]:
#Narrow the number of cities that meet the temperature, cloudiness and wind speed requirement
vacationcities = vacationweatherforcast.loc[(vacationweatherforcast["Temp"] > 70) & (vacationweatherforcast["Temp"] < 80) & (vacationweatherforcast["Cloudiness"] == 0) & (vacationweatherforcast["Winds Speed"] < 10), :]
#Eliminates null values using drpna
vacationcitiesfinal = vacationcities.dropna(how="any")
#reset the dataframe generated from the begining
vacationcitiesfinal.reset_index(inplace=True)
#delete index
del vacationcitiesfinal["index"]
vacationcitiesfinal.head(12) #test succesfully data frame generated

,City,Cloudiness,Country,Date,Humidity,Latitude,Longitude,Temp,Winds Speed
0,Port Alfred,0,ZA,1585581028,88,-33.59,26.89,71.01,5.99
1,Nyaunglebin,0,MM,1585581290,74,17.95,96.73,77.83,4.45
2,Chui,0,UY,1585581342,64,-33.70,-53.46,78.91,6.96
3,Sawākin,0,SD,1585581348,53,19.11,37.33,78.40,9.95
4,Tura,0,IN,1585581355,20,25.52,90.22,72.70,2.80
5,Pathein,0,MM,1585581364,91,16.78,94.73,75.87,6.49
6,Buraidah,0,SA,1585581409,31,26.33,43.98,75.20,5.82


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [24]:
#create hotel df variable
hotels_df = []
#create a loop to track down the number of hotels from gmaps url into the Dataframe  using latitude, Longitude and locations, using try/except function
for i in range(len(vacationcitiesfinal)):
    Latitude1 = vacationcitiesfinal.loc[i]["Latitude"]
    Longitude1 = vacationcitiesfinal.loc[i]["Longitude"]
    params = {
        "location": f"{Latitude1},{Longitude1}",
        "radius": 5000,
        "types" : "hotel",
        "key": g_key
    }
    hotels = requests.get(url, params=params)
    jsn = hotels.json()
    try:
        hotels_df.append(jsn['results'][0]['name'])
    except:
        hotels_ddf.append("")
vacationcitiesfinal["Hotel Name"] = hotels_df
#Eliminates null values using drpna
vacationcitiesfinal = vacationcitiesfinal.dropna(how='any')
vacationcitiesfinal.head(13) #test succesfully new data frame generated

,City,Cloudiness,Country,Date,Humidity,Latitude,Longitude,Temp,Winds Speed,Hotel Name
0,Port Alfred,0,ZA,1585581028,88,-33.59,26.89,71.01,5.99,Port Alfred
1,Nyaunglebin,0,MM,1585581290,74,17.95,96.73,77.83,4.45,Nyaung Lay Pin
2,Chui,0,UY,1585581342,64,-33.70,-53.46,78.91,6.96,Chuy
3,Sawākin,0,SD,1585581348,53,19.11,37.33,78.40,9.95,Suakin
4,Tura,0,IN,1585581355,20,25.52,90.22,72.70,2.80,Tura
5,Pathein,0,MM,1585581364,91,16.78,94.73,75.87,6.49,Pathein
6,Buraidah,0,SA,1585581409,31,26.33,43.98,75.20,5.82,Buraydah


In [26]:
# Generate new data frame with just the City, Hotel Name, Humidity, Latitude and Longitude
vacationcitiesfinalf = vacationcitiesfinal[["City","Hotel Name","Humidity","Latitude","Longitude","Winds Speed"]]
vacationcitiesfinalf.head(13)#test succesfully new data frame generated

,City,Hotel Name,Humidity,Latitude,Longitude,Winds Speed
0,Port Alfred,Port Alfred,88,-33.59,26.89,5.99
1,Nyaunglebin,Nyaung Lay Pin,74,17.95,96.73,4.45
2,Chui,Chuy,64,-33.70,-53.46,6.96
3,Sawākin,Suakin,53,19.11,37.33,9.95
4,Tura,Tura,20,25.52,90.22,2.80
5,Pathein,Pathein,91,16.78,94.73,6.49
6,Buraidah,Buraydah,31,26.33,43.98,5.82


In [27]:
# NOTE: Do not change any of the code in this cell
# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in vacationcitiesfinal.iterrows()]
locations = vacationcitiesfinal[["Latitude","Longitude" ]]

In [28]:
#Track down the Hotel in the city and country from the dataframe into gmaps using markers
markers = gmaps.marker_layer(locations)
Heatmap_layer.add_layer(markers)
print(f"Hotels recommeded to stay")
Heatmap_layer #test succesfully gmaps with markers generated

Hotels recommeded to stay


Figure(layout=FigureLayout(height='420px'))